# Automatic iOS photo and video curation and archiving from its iTunes backup files

## Asset types


| Kind | Kind subtype | PLAYBACK STYLE | PLAYBACK VARIATION | UNIFORM TYPE IDENTIFIER    | Description              |
|------|--------------|----------------|--------------------|----------------------------|--------------------------|
| 0    | 0            | 1              | 0                  | public.jpeg or public.heic | Photo                    |
| 0    | 0            | 1              | 0                  | public.jpeg or public.heic | Portrait                 |
| 0    | 10           | 1              | 0                  | public.png                 | Screenshot               |
| 0    | 2            | 3              | 0                  | public.jpeg or public.heic | Live Photo               |
| 0    | 2            | 5              | 1                  | public.jpeg or public.heic | Live Photo loop          |
| 0    | 2            | 5              | 2                  | public.jpeg or public.heic | Live Photo bounce        |
| 0    | 2            | 3              | 3                  | public.jpeg or public.heic | Live Photo long exposure |
| 1    | 0            | 4              | 0                  | com.apple.quicktime-movie  | Video                    |
| 1    | 101          | 4              | 0                  | com.apple.quicktime-movie  | Slowmotion               |
| 1    | 102          | 4              | 0                  | com.apple.quicktime-movie  | Timelapse                |
| 1    | 103          | 4              | 0                  | public.mpeg-4              | Screen recording         |

Following query on iOS 14’s `CameraRollDomain~Media/PhotoData/Photos.sqlite` will provide a good amount of data about assets:

```sql
 select
    ZASSET.Z_PK as Asset_PK,
    ZASSET.ZDIRECTORY as dcim_folder,
    ZASSET.ZFILENAME as file,
    ZASSETDESCRIPTION.ZLONGDESCRIPTION as caption,
    ZASSET.ZTRASHEDSTATE as trashed,
    ZASSET.ZHASADJUSTMENTS as edited,                
    case
        when ZASSET.ZKIND=0 then case -- images
            when ZASSET.ZKINDSUBTYPE=2 then '0>'||ZASSET.ZKINDSUBTYPE||'>'||ZASSET.ZPLAYBACKSTYLE||'>'||ZASSET.ZPLAYBACKVARIATION
            when ZASSET.ZPLAYBACKSTYLE=2 then '0>'||ZASSET.ZKINDSUBTYPE||'>'||ZASSET.ZPLAYBACKSTYLE
            else '0>'||ZASSET.ZKINDSUBTYPE
        end
        when ZASSET.ZKIND=1 then ZASSET.ZKIND||'>'||ZASSET.ZKINDSUBTYPE --videos
    end as kind_encoded,
    case 
        when ZASSET.ZKIND=0 then case -- images
            when ZASSET.ZKINDSUBTYPE=0 then case
                when ZASSET.ZPLAYBACKSTYLE=2 then 'animated GIF'
                else 'image' -- check ZCREATORBUNDLEID if its a PNG
            end
            when ZASSET.ZKINDSUBTYPE=10 then 'screenshot' -- created by this device
            when ZASSET.ZKINDSUBTYPE=2 then case -- live photos
                when ZASSET.ZPLAYBACKSTYLE=3 then case
                    when ZASSET.ZPLAYBACKVARIATION=0 then 'live photo'
                    when ZASSET.ZPLAYBACKVARIATION=3 then 'long exposure from live photo'
                end
                when ZASSET.ZPLAYBACKSTYLE=5 then case
                    when ZASSET.ZPLAYBACKVARIATION=1 then 'live photo, loop effect'
                    when ZASSET.ZPLAYBACKVARIATION=2 then 'live photo, bounce effect'
                end
            end
        end
        when ZASSET.ZKIND=1 then case -- videos
            when ZASSET.ZKINDSUBTYPE=0 then 'video'
            when ZASSET.ZKINDSUBTYPE=101 then 'slow motion video'
            when ZASSET.ZKINDSUBTYPE=102 then 'timelapse video'
            when ZASSET.ZKINDSUBTYPE=103 then 'screencast'
        end
    end as kind_description,
    ZASSET.ZFAVORITE as favorited,
    ZASSET.ZWIDTH as width,
    ZASSET.ZHEIGHT as height,
    ZASSET.ZDURATION as video_duration,
    ZASSET.ZUUID as uuid,
    ZMOMENT.ZTITLE as moment_title,
    ZMOMENT.ZSUBTITLE as moment_subtitle,
    ZASSET.ZDATECREATED as creation_timestamp,
    datetime(ZASSET.ZDATECREATED+strftime('%s','2001-01-01'),'unixepoch') as utc_time,
    ZADDITIONALASSETATTRIBUTES.ZINFERREDTIMEZONEOFFSET as tz_offset,
    datetime(ZASSET.ZDATECREATED+ZADDITIONALASSETATTRIBUTES.ZINFERREDTIMEZONEOFFSET+strftime('%s','2001-01-01'),'unixepoch') as asset_local_time,
    ZADDITIONALASSETATTRIBUTES.ZEXIFTIMESTAMPSTRING as exif_timestamp,
    ZADDITIONALASSETATTRIBUTES.ZREVERSELOCATIONDATA as location_data,
    facecount.facecount,
    named_facecount.named_facecount
from
    ZASSET

    left outer join ZADDITIONALASSETATTRIBUTES
        on ZADDITIONALASSETATTRIBUTES.ZASSET=ZASSET.Z_PK

    left outer join ZASSETDESCRIPTION
        on ZASSETDESCRIPTION.ZASSETATTRIBUTES=ZADDITIONALASSETATTRIBUTES.Z_PK

    left outer join ZMOMENT
        on ZMOMENT.Z_PK=ZASSET.ZMOMENT

    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as facecount
        from ZDETECTEDFACE
        group by ZASSET
    ) as facecount
        on facecount.asset=ZASSET.Z_PK

    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as named_facecount
        from ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
        group by ZASSET
    ) as named_facecount
        on named_facecount.asset=ZASSET.Z_PK

where
    ZASSET.ZDIRECTORY like 'DCIM%'
    
order by ZASSET.ZDATECREATED desc
```


In [ ]:
import sys
# sys.path.insert(0,"..") # Adds higher directory to python modules path.

import pprint
import logging
import datetime
import math
from PhotoCuration import *

# logging.basicConfig(level=logging.DEBUG)

In [ ]:
engine=PhotoCuration(
    start=datetime.datetime(2020,2,19,12,44),
#     end=datetime.datetime(2020,10,27),
    author='Avi Alkalay',
#     originals=True,
#     extractTypes=['0>0', '0>0>2', '0>10', '0>0>1>2'],
#     extractTypes=['1>0', '1>101', '1>102', '1>103'],
#     extractTypes=['1>101'],
#     target='/media/sf_aviram/Notebooks/iOSbackup/playground/aa-uu-2'
    target='aa-uu-2'
)

In [ ]:
%%time
engine.fetchAssets()
engine.fetchAlbums()
engine.fetchMemories()
engine.fetchPeople()
engine.calcAssetsKeywords()
engine.fetchPlacesMoments()
engine.addAssetNameFromSmallestAlbum()
engine.addAssetNameFromSmallestMemory()
engine.calcBestLocationName()
engine.calcBestAssetCaption()

In [ ]:
# %%time
engine.extractAndTag()

In [ ]:
engine.report()

In [ ]:
engine.assets.columns

In [ ]:
engine.assetCurrent['spawn']['main']['tags']['collision_index']

In [ ]:
engine.assets[engine.assets['file']=='IMG_8654.MOV']

In [ ]:
del engine

In [ ]:
import pathlib
p=str(pathlib.PurePath('a','b'))
p

In [ ]:
x={'a':1, 'b':2}

In [ ]:
engine.assets.columns

In [ ]:
engine.assets.info() #['asset_local_time'].info()

In [ ]:
aaa=engine.assets[['dcim_folder', 'file','kind', 'favorited','width', 'height', 'video_duration','uuid','location_suggested_name','location_name', 'location_context', 'location_street',
       'location_subLocality', 'location_city', 'location_adminArea',
       'location_state', 'location_country', 'location_postalCode',
       'location_countryCode', 'location_formattedAddress']].copy()

In [ ]:
aaa['suggested_caption']=engine.assets['infered_asset_caption']
aaa['suggested_caption_score']=engine.assets['infered_asset_caption_score']

In [ ]:
items=[
    ('%Y.%m.%d-%H.%M.%S', 'time_fs'),
    ('Y', 'year'),
    ('m', 'month'),
    ('d', 'day'),
    ('H', 'hour'),
    ('M', 'minute'),
    ('S', 'second'),
    ('y', 'year_small'),
    ('I', 'hour12'),
    ('p', 'ampm'),
    ('b', 'month_name_abrev'),
    ('B', 'month_name'),
    ('a', 'weekday_abrev'),
    ('A', 'weekday'),
    ('c', 'locale_time')
]

dd=pd.DataFrame()
prefix='creation_local'

for i in items:
    if '%' in i[0]:
        format=i[0]
    else:
        format=f'%{i[0]}'
    dd[f'{prefix}_{i[1]}']=engine.assets.asset_local_time.dt.strftime(format)

In [ ]:
dd=['dcim_folder', 'file','kind', 'favorited','width', 'height', 'video_duration','uuid','location_suggested_name','location_name', 'location_context', 'location_street',
       'location_subLocality', 'location_city', 'location_adminArea',
       'location_state', 'location_country', 'location_postalCode',
       'location_countryCode', 'location_formattedAddress']

In [ ]:
dd

In [ ]:
engine.assets.columns

In [ ]:
items

In [ ]:
aaa

In [ ]:
engine.extractAndTag()

In [ ]:
engine.assets[engine.assets['trashed']==0].sort_values(by='utc_time').columns

In [ ]:
engine.assets[['utc_time','asset_local_time']].info()

In [ ]:
pd.set_option('display.max_rows', None)
engine.report()
# engine.assets[engine.assets['infered_asset_caption_score']<0.5][['utc_time','tz_offset','asset_local_time','kind','video_duration','infered_asset_caption','location_suggested_name']]
# engine.assets[['trashed','infered_asset_caption','infered_asset_caption_score','location_suggested_name','memory_title','album_title','caption']].drop_duplicates()

In [ ]:
engine.locations.location_name.value_counts()

In [ ]:
engine.assets[True]

In [ ]:
import jinja2

In [ ]:
template="""
{{localtime}} {% if caption -%}
{% if video -%}
▶️ {# -#}
{% else -%}
{% if favorited -%}
★ {# -#}
{% else -%}
• {#+ -#}
{% endif -%}
{% endif -%}
{{ caption }} {# -#}
{% endif -%}
{% if author or device -%}
【{{author}}{# -#}
{% if author and device %}·︎{% endif -%}
{{device}}】{# -#}
{% endif -%}
.{{ext}}
"""

In [ ]:
l=list(engine.assets['Moment_Title'].value_counts().index)
l.sort()
l

In [ ]:
l=list(engine.locations['location_name'].value_counts().index)
l.sort()
l

In [ ]:
engine.locations[engine.locations['location_name'] == 'Ale’s home']

In [ ]:
engine.assets.columns

In [ ]:
engine.assetsOfMemories.join(engine.assetsOfMemories['memory'].value_counts(), on='memory', rsuffix='memory_asset_count')

In [ ]:
engine.memories

In [ ]:
engine.assetsOfMemories.set_index('memory').join(engine.memories)[['memory_uuid','memory_title','assetUUID']]#.sort_values('album_asset_count').groupby('assetUUID').head(1).set_index('assetUUID')


In [ ]:
engine.assets

In [ ]:
engine.assets[['dcim_folder','file','favorited','uuid','Moment_Title','album_title']]#['album_title'].value_counts()

In [ ]:
engine.assetsOfAlbums.set_index('album').join(engine.albums)[['album_uuid','album_title','album_asset_count','assetUUID']].sort_values('album_asset_count').groupby('assetUUID').head(1).set_index('assetUUID')


In [ ]:
engine.assetsOfAlbums

In [ ]:
rel=engine.assetsOfAlbums.set_index('album').join(engine.albums)[['album_uuid','album_title','album_asset_count','assetUUID']]
rel.groupby('assetUUID')['album_asset_count']

In [ ]:
rel.sort_values('album_asset_count').groupby('assetUUID').head(1)

In [ ]:
engine.assetsOfAlbums.set_index('album').join(engine.albums)[['album_uuid','album_title','album_asset_count','assetUUID']].sort_values('album_asset_count').groupby('assetUUID').head(1)

In [ ]:
engine.assets

In [ ]:
Index(['albumUUID', 'albumTrashed', 'assetUUID', 'album_uuid', 'album_title',
       'album_public_url', 'album_asset_count', 'album_photo_count',
       'album_video_count', 'album_cloud_deleted_state',
       'album_cloud_local_state', 'album_custom_sort_ascending',
       'album_custom_sort_key', 'ZKIND', 'ZPENDINGITEMSCOUNT',
       'ZPENDINGITEMSTYPE', 'ZSYNCEVENTORDERKEY', 'album_is_trashed',
       'ZCUSTOMKEYASSET', 'ZKEYASSET', 'ZSECONDARYKEYASSET',
       'ZTERTIARYKEYASSET', 'ZCLOUDALBUMSUBTYPE',
       'ZCLOUDMULTIPLECONTRIBUTORSENABLED',
       'ZCLOUDMULTIPLECONTRIBUTORSENABLEDLOCAL', 'ZCLOUDNOTIFICATIONSENABLED',
       'ZCLOUDOWNEREMAILKEY', 'ZCLOUDOWNERISWHITELISTED',
       'ZCLOUDPUBLICURLENABLED', 'ZCLOUDPUBLICURLENABLEDLOCAL',
       'ZCLOUDRELATIONSHIPSTATE', 'ZCLOUDRELATIONSHIPSTATELOCAL',
       'ZHASUNSEENCONTENT', 'ZISOWNED', 'ZUNSEENASSETSCOUNT',
       'ZKEYASSETFACEIDENTIFIER', 'ZKEYASSETFACETHUMBNAILINDEX',
       'album_parent_folder', 'album_parent_folder_pk', 'album_utc_creation',
       'album_utc_start', 'album_utc_end', 'ZTRASHEDDATE',
       'ZCLOUDCREATIONDATE', 'ZCLOUDLASTCONTRIBUTIONDATE',
       'ZCLOUDLASTINTERESTINGCHANGEDATE', 'ZCLOUDSUBSCRIPTIONDATE',
       'ZCLOUDGUID', 'ZIMPORTSESSIONID', 'ZCLOUDOWNERFIRSTNAME',
       'ZCLOUDOWNERFULLNAME', 'ZCLOUDOWNERHASHEDPERSONID',
       'ZCLOUDOWNERLASTNAME', 'ZCLOUDPERSONID', 'ZPROJECTDOCUMENTTYPE',
       'ZPROJECTEXTENSIONIDENTIFIER', 'ZPROJECTRENDERUUID',
       'ZCUSTOMQUERYTYPE'],
      dtype='object')

In [ ]:
engine.assetsOfAlbums.groupby(by=['assetUUID'])

In [ ]:
a=list(engine.assets2.columns)
a.sort()
a

In [ ]:
import numpy as np

engine.assets2[[
    'location_name',
 'location_context',
 'location_street',
 'location_subLocality',
 'location_city',
'location_adminArea',
 'location_state',
 'location_country',
 'location_postalCode',
 'location_countryCode',
 'location_formattedAddress',
]]

In [ ]:
engine.assets2.loc[0]['location_postalCode']

In [ ]:
engine.memories.iloc[12]

In [ ]:
import biplist

phdata=biplist.readPlistFromString(engine.memories.iloc[12]['ZMOVIEASSETSTATE'])

In [ ]:
phdata

In [ ]:
import biplist

In [ ]:
f='/Users/avi/Notebooks/iOSbackup/playground/aa-uu/CameraRollDomain/Media/PhotoData/moments/ZADDITIONALASSETATTRIBUTES-17356.plist'
data=biplist.readPlist(f)

In [ ]:
mapItem_index=data['$objects'][data['$top']['root'].integer]['mapItem'].integer

sortedPlaceInfos_index=data['$objects'][mapItem_index]['sortedPlaceInfos'].integer

place_index=data['$objects'][sortedPlaceInfos_index]['NS.objects'][0].integer

placeName_index=data['$objects'][place_index]['name'].integer

data['$objects'][placeName_index]

In [ ]:
place=[]

for l in data['$objects'][sortedPlaceInfos_index]['NS.objects']:
    placeName_index=data['$objects'][l.integer]['name'].integer
    place.append(data['$objects'][placeName_index])
    
places=set()
places_add=places.add
a=[x for x in place if not (x in places or places_add(x))]

In [ ]:
' < '.join(a)

In [ ]:
data

In [ ]:
data['$objects'][1]['mapItem']['CF$UID']

In [ ]:
data

In [ ]:
import sys
sys.path.insert(0,"/Users/avi/src/ccl-bplist") # Adds higher directory to python modules path.

import ccl_bplist

ccl_bplist.set_object_converter(ccl_bplist.NSKeyedArchiver_common_objects_convertor)

In [ ]:
f='/Users/avi/Notebooks/iOSbackup/playground/aa-uu/CameraRollDomain/Media/PhotoData/moments/ZADDITIONALASSETATTRIBUTES-17356.plist'
d=ccl_bplist.deserialise_NsKeyedArchiver(ccl_bplist.load(open(f,"rb")))

In [ ]:
d #['mapItem']['sortedPlaceInfos'][0]['name']

In [ ]:
Z_PK,
Z_ENT,
Z_OPT,
ZALLOWEDFORANALYSIS,
ZCAMERACAPTUREDEVICE,
ZCLOUDAVALANCHEPICKTYPE,
ZCLOUDKINDSUBTYPE,
ZCLOUDRECOVERYSTATE,
ZCLOUDSTATERECOVERYATTEMPTSCOUNT,
ZDEFERREDPROCESSINGCANDIDATEOPTIONS,
ZDESTINATIONASSETCOPYSTATE,
ZEMBEDDEDTHUMBNAILHEIGHT,
ZEMBEDDEDTHUMBNAILLENGTH,
ZEMBEDDEDTHUMBNAILOFFSET,
ZEMBEDDEDTHUMBNAILWIDTH,
ZIMPORTEDBY,
ZINFERREDTIMEZONEOFFSET,
ZLOCATIONHASH,
ZORIGINALFILESIZE,
ZORIGINALHEIGHT,
ZORIGINALORIENTATION,
ZORIGINALRESOURCECHOICE,
ZORIGINALWIDTH,
ZPENDINGPLAYCOUNT,
ZPENDINGSHARECOUNT,
ZPENDINGVIEWCOUNT,
ZPLAYCOUNT,
ZPTPTRASHEDSTATE,
ZREVERSELOCATIONDATAISVALID,
ZSCENEANALYSISVERSION,
ZSHARECOUNT,
ZSHARETYPE,
ZSHIFTEDLOCATIONISVALID,
ZTIMEZONEOFFSET,
ZUPLOADATTEMPTS,
ZVARIATIONSUGGESTIONSTATES,
ZVIDEOCPDISPLAYTIMESCALE,
ZVIDEOCPDISPLAYVALUE,
ZVIDEOCPDURATIONTIMESCALE,
ZVIEWCOUNT,
ZASSET,
ZASSETDESCRIPTION,
ZEDITEDIPTCATTRIBUTES,
ZMEDIAMETADATA,
ZSCENEPRINT,
ZUNMANAGEDADJUSTMENT,
ZALTERNATEIMPORTIMAGEDATE,
ZGPSHORIZONTALACCURACY,
ZLASTUPLOADATTEMPTDATE,
ZSCENEANALYSISTIMESTAMP,
ZACCESSIBILITYDESCRIPTION,
ZADJUSTEDFINGERPRINT,
ZCREATORBUNDLEID,
ZDEFERREDPHOTOIDENTIFIER,
ZEDITORBUNDLEID,
ZEXIFTIMESTAMPSTRING,
ZIMPORTSESSIONID,
ZMASTERFINGERPRINT,
ZMEDIAMETADATATYPE,
ZMONTAGE,
ZORIGINALASSETSUUID,
ZORIGINALFILENAME,
ZORIGINATINGASSETIDENTIFIER,
ZPHOTOSTREAMTAGID,
ZPUBLICGLOBALUUID,
ZSHAREORIGINATOR,
ZSNOWDAYSNOWPLOWIDENTIFIER,
ZSPATIALOVERCAPTUREGROUPIDENTIFIER,
ZTIMEZONENAME,
ZTITLE,
ZDISTANCEIDENTITY,
ZFACEREGIONS,
ZOBJECTSALIENCYRECTSDATA,
ZORIGINALHASH,
ZPLACEANNOTATIONDATA,
ZREVERSELOCATIONDATA,
ZSHIFTEDLOCATIONDATA

In [ ]:
engine.assetsOfMemories

In [ ]:
import uuid

In [ ]:
x=uuid.UUID('7aec8921-5353-4c8d-bc8d-a92da5258e0c')

In [ ]:
str(x).upper()

In [ ]:
engine.albums.info()

In [ ]:
data['$objects'][data['$top']['assetUUIDs'].integer]

In [ ]:
import plistlib
import biplist

In [ ]:
data=b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07\x1cX$versionY$archiverT$topX$objects\x12\x00\x01\x86\xa0_\x10\x0fNSKeyedArchiver\xda\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x12\x14\x15\x16\x17\x12\x19\x12\x1b[isPrototype]customSortKey_\x10\x13customSortAscendingZassetUUIDsUtitleTuuidXisPinnedTkindYisInTrashWversion\x80\x04\x80\x04\x80\x05\x80\x06\x80\x01\x80\x03\x80\x04\x80\x02\x80\x04\x10\x01\xa7\x1d\x1e\x1f !"#U$nullYInstagram\x10\x02_\x10$12D64593-9CD0-45AD-AFB1-D1F29FB27240\x10\x00\x10\x01O\x10\xe0z\xec\x89!SSL\x8d\xbc\x8d\xa9-\xa5%\x8e\x0c\xfe\xdar\x17\xcdWHZ\x90\xe5\x95+euq\x15\x02\x1eR\x9b\t\x9fK\xd5\x94\xfc\xbd\n\xde\xae\xb4\xadP\x11>>\x16\xd3I;\x89\x94P\x0b\x8b\xe0Ek]+\xb9\xb8\x882@e\xbd\xf0\xe3\x18\xc1\x88(qI\xb4\x19\xc0\xf9uE\xea\xa0(Nm\x97Y\x9b\x8f\x1d\xfd\x9eBM\xcaH\x9e\xa3\xbe\xb1\xa3A\xa28S\xe0bk\xc6\x02\x18Gi\xa1\xb5#/\xe2d\xc5+H\x07\xae\xff6BI(\xa9\xb5\xf1\x8c\xf7l\xd8)\xe6^\xbe[\xfb\x0cD\x03\x97<\x0e\x91\x8f\xaa\t\xfc\x0cY\x96x\xbf\xb0K\x9d\x83\x91\xe9\x86\xeaq\x1e1\xd2\xff\x0f&\x1c\xcbH\xa4\x8eE\xc5\xdb\xaa\xa1:\xa4N\xd6\xc6\xf9\xc8FA\xe0\x92;\xe7\x90\x19~\x1f\xa4\x8c\x0b\xc8D\xa3fNm\x8f\x8f\xe2n\xb9\xc5\x19\xf4\x00\x08\x00\x11\x00\x1a\x00$\x00)\x002\x007\x00I\x00^\x00j\x00x\x00\x8e\x00\x99\x00\x9f\x00\xa4\x00\xad\x00\xb2\x00\xbc\x00\xc4\x00\xc6\x00\xc8\x00\xca\x00\xcc\x00\xce\x00\xd0\x00\xd2\x00\xd4\x00\xd6\x00\xd8\x00\xe0\x00\xe6\x00\xf0\x00\xf2\x01\x19\x01\x1b\x01\x1d\x00\x00\x00\x00\x00\x00\x02\x01\x00\x00\x00\x00\x00\x00\x00$\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00'
# a=plistlib.loads(data,fmt=plistlib.FMT_BINARY)

a=biplist.readPlistFromString(data)

In [ ]:
a

In [ ]:
a['$top']['uuid'].integer

In [ ]:
engine.getPhotoData()

In [ ]:
engine.assets.info()

In [ ]:
engine.assets

In [ ]:
engine.faces.info()

In [ ]:
engine.faces

In [ ]:
engine.processPhotos()

In [ ]:
del engine

In [ ]:
engine.assets[engine.assets['favorited']!=0]

In [ ]:
engine.iosDBs

In [ ]:
engine.sync()

In [ ]:
del engine

## All data about photos

```sql
select
    ZGENERICASSET.Z_PK as Asset_PK,
    ZGENERICASSET.ZDIRECTORY,
    ZGENERICASSET.ZFILENAME,
    ZGENERICASSET.ZTRASHEDSTATE,
    ZGENERICASSET.ZFAVORITE,
    ZGENERICASSET.ZWIDTH,
    ZGENERICASSET.ZHEIGHT,
    ZMOMENT.ZTITLE as Moment_Title,
    ZMOMENT.ZSUBTITLE as Moment_Subtitle,
    datetime(ZGENERICASSET.ZDATECREATED+strftime('%s','2001-01-01','utc'),'unixepoch') as taken,
    ZADDITIONALASSETATTRIBUTES.ZTIMEZONEOFFSET,
    ZADDITIONALASSETATTRIBUTES.ZEXIFTIMESTAMPSTRING,
    facecount.facecount,
    named_facecount.named_facecount,
    people.Detected_PK,
    people.Person_PK,
    people.short_name,
    people.full_name,
    people.adjustment_ver,
    people.width,
    people.height,
    people.LEFTEYE_pixx,
    people.LEFTEYE_pixy,
    people.RIGHTEYE_pixx,
    people.RIGHTEYE_pixy,
    people.MOUTH_pixx,
    people.MOUTH_pixy,
    people.CENTER_pixx,
    people.CENTER_pixy,
    people.face_size,
    people.person_uri,
    people.person_uuid
from
    ZGENERICASSET
    
    left outer join ZADDITIONALASSETATTRIBUTES
        on ZADDITIONALASSETATTRIBUTES.ZASSET=ZGENERICASSET.Z_PK
        
    left outer join ZMOMENT
        on ZMOMENT.Z_PK= ZGENERICASSET.ZMOMENT
        
    left outer join (
        select 
            ZDETECTEDFACE.Z_PK as Detected_PK,
            ZPERSON.Z_PK as Person_PK,
            ZPERSON.ZDISPLAYNAME as short_name,
            ZPERSON.ZFULLNAME as full_name,
            ZDETECTEDFACE.ZADJUSTMENTVERSION as adjustment_ver,
            ZDETECTEDFACE.ZSOURCEWIDTH as width,
            ZDETECTEDFACE.ZSOURCEHEIGHT as height,
            cast(round(ZDETECTEDFACE.ZLEFTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) LEFTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZLEFTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) LEFTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) RIGHTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) RIGHTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZMOUTHX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) MOUTH_pixx,
            cast(round(ZDETECTEDFACE.ZMOUTHY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) MOUTH_pixy,
            cast(round(ZDETECTEDFACE.ZCENTERX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) CENTER_pixx,
            cast(round(ZDETECTEDFACE.ZCENTERY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) CENTER_pixy,
            ZDETECTEDFACE.ZSIZE as face_size, -- face_size×√(width²+height²) = diameter of circle surrounding face
            ZPERSON.ZPERSONURI as person_uri,
            ZPERSON.ZPERSONUUID as person_uuid,
            ZDETECTEDFACE.ZASSET as asset
        from
            ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
    ) as people
        on people.asset=ZGENERICASSET.Z_PK
    
    
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as facecount
        from ZDETECTEDFACE
        group by ZASSET
    ) as facecount
        on facecount.asset=ZGENERICASSET.Z_PK
    
    
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as named_facecount
        from ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
        group by ZASSET
    ) as named_facecount
        on named_facecount.asset=ZGENERICASSET.Z_PK
    
    
where
    ZGENERICASSET.ZDIRECTORY like 'DCIM%'
--  and Person_PK=5

order by
        taken, Detected_PK
```



## List of Assets


```sql
select
    ZGENERICASSET.Z_PK as Asset_PK,
    ZGENERICASSET.ZDIRECTORY,
    ZGENERICASSET.ZFILENAME,
    ZGENERICASSET.ZTRASHEDSTATE,
    ZGENERICASSET.ZFAVORITE,
    ZGENERICASSET.ZWIDTH,
    ZGENERICASSET.ZHEIGHT,
    ZMOMENT.ZTITLE as Moment_Title,
    ZMOMENT.ZSUBTITLE as Moment_Subtitle,
    datetime(ZGENERICASSET.ZDATECREATED+strftime('%s','2001-01-01','utc'),'unixepoch') as utc_time,
    ZADDITIONALASSETATTRIBUTES.ZTIMEZONEOFFSET,
    ZADDITIONALASSETATTRIBUTES.ZEXIFTIMESTAMPSTRING,
    facecount.facecount,
    named_facecount.named_facecount
from
    ZGENERICASSET
    
    left outer join ZADDITIONALASSETATTRIBUTES
        on ZADDITIONALASSETATTRIBUTES.ZASSET=ZGENERICASSET.Z_PK
        
    left outer join ZMOMENT
        on ZMOMENT.Z_PK= ZGENERICASSET.ZMOMENT
        
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as facecount
        from ZDETECTEDFACE
        group by ZASSET
    ) as facecount
        on facecount.asset=ZGENERICASSET.Z_PK
    
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as named_facecount
        from ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
        group by ZASSET
    ) as named_facecount
        on named_facecount.asset=ZGENERICASSET.Z_PK
    
where
    ZGENERICASSET.ZDIRECTORY like 'DCIM%'
--  and Person_PK=5
```



## Faces for Asset

```sql
select
    ZGENERICASSET.Z_PK as Asset_PK,
    people.Detected_PK,
    people.Person_PK,
    people.short_name,
    people.full_name,
    people.adjustment_ver,
    people.width,
    people.height,
    people.LEFTEYE_pixx,
    people.LEFTEYE_pixy,
    people.RIGHTEYE_pixx,
    people.RIGHTEYE_pixy,
    people.MOUTH_pixx,
    people.MOUTH_pixy,
    people.CENTER_pixx,
    people.CENTER_pixy,
    people.face_size,
    people.person_uri,
    people.person_uuid
from
    (
        select 
            ZDETECTEDFACE.Z_PK as Detected_PK,
            ZPERSON.Z_PK as Person_PK,
            ZPERSON.ZDISPLAYNAME as short_name,
            ZPERSON.ZFULLNAME as full_name,
            ZDETECTEDFACE.ZADJUSTMENTVERSION as adjustment_ver,
            ZDETECTEDFACE.ZSOURCEWIDTH as width,
            ZDETECTEDFACE.ZSOURCEHEIGHT as height,
            cast(round(ZDETECTEDFACE.ZLEFTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) LEFTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZLEFTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) LEFTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) RIGHTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) RIGHTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZMOUTHX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) MOUTH_pixx,
            cast(round(ZDETECTEDFACE.ZMOUTHY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) MOUTH_pixy,
            cast(round(ZDETECTEDFACE.ZCENTERX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) CENTER_pixx,
            cast(round(ZDETECTEDFACE.ZCENTERY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) CENTER_pixy,
            ZDETECTEDFACE.ZSIZE as face_size, -- face_size×√(width²+height²) = diameter of circle surrounding face
            ZPERSON.ZPERSONURI as person_uri,
            ZPERSON.ZPERSONUUID as person_uuid,
            ZDETECTEDFACE.ZASSET as asset
        from
            ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
    ) as people
    
    left outer join ZGENERICASSET
        on ZGENERICASSET.Z_PK=people.asset
    
where
    ZGENERICASSET.ZDIRECTORY like 'DCIM%'
--  and Person_PK=5

order by
        Asset_PK desc
```




In [ ]:
x=[1,3,2]
sorted(x)